# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up graphlab create

In [8]:
import graphlab
import math

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to dxiao@bloomu.edu and will expire on April 02, 2020.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\danqi\AppData\Local\Temp\graphlab_server_1558059911.log.0


In [11]:
sales.head()

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650,1,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242,2,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000,1,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000,1,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080,1,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930,1,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819,2,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711,1,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470,1,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560,2,0


# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

In [4]:
train_data,test_data = sales.random_split(.8,seed=0)

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [5]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
example_model = graphlab.linear_regression.create(train_data, target = 'price', features = example_features, 
                                                  validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 3

Number of unpacked features : 3

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 1.014167     | 4146407.600631     | 258679.804477 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [ ]:
example_weight_summary = example_model.get("coefficients")
print example_weight_summary

# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing graphlab create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [ ]:
example_predictions = example_model.predict(train_data)
print example_predictions[0] # should be 271789.505878

# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [ ]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions

    # Then compute the residuals/errors

    # Then square and add them up

    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [ ]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
print rss_example_train # should be 2.7376153833e+14

# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [ ]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [14]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [6]:
# create the remaining 3 features in both TEST and TRAIN data
train_data['bed_bath_rooms'] = train_data['bedrooms']* train_data['bathrooms']
test_data['bed_bath_rooms'] = test_data['bedrooms']* test_data['bathrooms']


In [10]:
train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: math.log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: math.log(x))

In [45]:
train_data['lat_plus_long'] = train_data['lat']+ train_data['long']
test_data['lat_plus_long'] = test_data['lat']+test_data['long']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

In [16]:
test_data['bedrooms_squared'].sketch_summary()


+--------------------+---------------+----------+
|        item        |     value     | is exact |
+--------------------+---------------+----------+
|       Length       |      4229     |   Yes    |
|        Min         |      0.0      |   Yes    |
|        Max         |     1089.0    |   Yes    |
|        Mean        | 12.4466777016 |   Yes    |
|        Sum         |    52637.0    |   Yes    |
|      Variance      | 319.224692793 |   Yes    |
| Standard Deviation | 17.8668601828 |   Yes    |
|  # Missing Values  |       0       |   Yes    |
|  # unique values   |       11      |    No    |
+--------------------+---------------+----------+

Most frequent items:
+-------+------+------+-----+------+------+-----+------+-----+------+--------+
| value | 9.0  | 16.0 | 4.0 | 25.0 | 36.0 | 1.0 | 49.0 | 0.0 | 64.0 | 1089.0 |
+-------+------+------+-----+------+------+-----+------+-----+------+--------+
| count | 1864 | 1369 | 561 | 329  |  49  |  43 |  7   |  3  |  2   |   1    |
+-------+--

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [17]:
test_data['bed_bath_rooms'].sketch_summary()


+--------------------+---------------+----------+
|        item        |     value     | is exact |
+--------------------+---------------+----------+
|       Length       |      4229     |   Yes    |
|        Min         |      0.0      |   Yes    |
|        Max         |     57.75     |   Yes    |
|        Mean        | 7.50390163159 |   Yes    |
|        Sum         |    31734.0    |   Yes    |
|      Variance      | 17.8635754606 |   Yes    |
| Standard Deviation | 4.22653232102 |   Yes    |
|  # Missing Values  |       0       |   Yes    |
|  # unique values   |       68      |    No    |
+--------------------+---------------+----------+

Most frequent items:
+-------+------+-----+-----+------+-----+-----+------+-----+-----+-----+
| value | 10.0 | 7.5 | 3.0 | 5.25 | 2.0 | 6.0 | 6.75 | 9.0 | 4.5 | 7.0 |
+-------+------+-----+-----+------+-----+-----+------+-----+-----+-----+
| count | 486  | 479 | 383 | 357  | 312 | 246 | 206  | 189 | 176 | 169 |
+-------+------+-----+-----+------+

In [18]:
test_data['log_sqft_living'].sketch_summary()


+--------------------+----------------+----------+
|        item        |     value      | is exact |
+--------------------+----------------+----------+
|       Length       |      4229      |   Yes    |
|        Min         | 5.91350300564  |   Yes    |
|        Max         | 9.19927942462  |   Yes    |
|        Mean        | 7.55027467965  |   Yes    |
|        Sum         | 31930.1116202  |   Yes    |
|      Variance      | 0.181905451212 |   Yes    |
| Standard Deviation | 0.42650375287  |   Yes    |
|  # Missing Values  |       0        |   Yes    |
|  # unique values   |      524       |    No    |
+--------------------+----------------+----------+

Most frequent items:
+-------+---------------+--------------+---------------+---------------+---------------+
| value | 7.50659178007 | 7.2442275156 | 7.35244110024 | 7.09007683578 | 7.57044325206 |
+-------+---------------+--------------+---------------+---------------+---------------+
| count |       34      |      32      |       

In [46]:
test_data['lat_plus_long'].sketch_summary()


+--------------------+-----------------+----------+
|        item        |      value      | is exact |
+--------------------+-----------------+----------+
|       Length       |       4229      |   Yes    |
|        Min         |   -75.17746438  |   Yes    |
|        Max         |   -73.60204149  |   Yes    |
|        Mean        |  -74.6533349722 |   Yes    |
|        Sum         |  -315708.953597 |   Yes    |
|      Variance      | 0.0331788941705 |   Yes    |
| Standard Deviation |  0.182150745731 |   Yes    |
|  # Missing Values  |        0        |   Yes    |
|  # unique values   |       4239      |    No    |
+--------------------+-----------------+----------+

Most frequent items:
+-------+--------------+--------------+--------------+--------------+-------------+
| value | -74.75153822 | -74.82562611 | -74.55128486 | -74.42048989 | -74.5979727 |
+-------+--------------+--------------+--------------+--------------+-------------+
| count |      2       |      2       |      2   

# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [23]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_1 = graphlab.linear_regression.create(train_data, target = 'price', features = model_1_features, 
                                                  validation_set = None)



Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 5

Number of unpacked features : 5

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.021822     | 4074878.213096     | 236378.596455 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

In [24]:
model_1.get("coefficients")

name,index,value,stderr
(intercept),None,-56140675.7444,1649985.42028
sqft_living,None,310.263325778,3.18882960408
bedrooms,None,-59577.1160683,2487.27977322
bathrooms,None,13811.8405419,3593.54213297
lat,None,629865.789485,13120.7100323
long,None,-214790.285186,13284.2851607


Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [22]:
# Learn the three models: (don't forget to set validation_set = None)
model_2_features = model_1_features + ['bed_bath_rooms']
model_2 = graphlab.linear_regression.create(train_data, target = 'price', features = model_2_features, 
                                                  validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 6

Number of unpacked features : 6

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.029266     | 4014170.932928     | 235190.935428 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

In [25]:
model_2.get("coefficients")

name,index,value,stderr
(intercept),None,-54410676.1152,1650405.16541
sqft_living,None,304.449298056,3.20217535637
bedrooms,None,-116366.04323,4805.54966546
bathrooms,None,-77972.3305131,7565.05991091
lat,None,625433.834953,13058.3530972
long,None,-203958.602959,13268.1283711
bed_bath_rooms,None,26961.6249091,1956.36561555


In [26]:
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']
model_3 = graphlab.linear_regression.create(train_data, target = 'price', features = model_3_features, 
                                                  validation_set = None)

Linear regression:

--------------------------------------------------------

Number of examples          : 17384

Number of features          : 9

Number of unpacked features : 9

...

| Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |

+-----------+----------+--------------+--------------------+---------------+

| 1         | 2        | 0.015383     | 3190659.608868     | 228020.710401 |

+-----------+----------+--------------+--------------------+---------------+

SUCCESS: Optimal solution found.

In [27]:
model_3.get("coefficients")

name,index,value,stderr
(intercept),None,-36948386.4768,555372704.576
sqft_living,None,528.730615806,7.69599810643
bedrooms,None,31840.0776555,9390.00407029
bathrooms,None,65878.6982786,10786.9158877
lat,None,223885.935051,11679479.6895
long,None,-78226.2325171,4544354.76968
bed_bath_rooms,None,-8411.39576291,2856.95194534
bedrooms_squared,None,-6466.62967114,1493.8649694
log_sqft_living,None,-562014.195443,17554.0700973
lat_plus_long,None,-3495.70971979,95567.4635201


In [ ]:
# Examine/extract each model's coefficients:


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [29]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
model_1.get('training_loss')


971328233543692.2

In [30]:
model_2.get('training_loss')

961592067855797.0

**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

In [31]:

model_3.get('training_loss')

903854036958014.5

Now compute the RSS on on TEST data for each of the three models.

In [41]:
# Compute the RSS on TESTING data for each of the three models and record the values:
test_data['1rss']=model_1.predict(test_data)
test_data['2rss']=model_2.predict(test_data)
test_data['3rss']=model_3.predict(test_data)
test_data['11rss']=(test_data['1rss']-test_data['price'])**2
test_data['22rss']=(test_data['2rss']-test_data['price'])**2
test_data['33rss']=(test_data['3rss']-test_data['price'])**2


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.

In [38]:
test_data.head()

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
0114101516,2014-05-28 00:00:00+00:00,310000.0,3.0,1.0,1430.0,19901,1.5,0
9297300055,2015-01-24 00:00:00+00:00,650000.0,4.0,3.0,2950.0,5000,2,0
1202000200,2014-11-03 00:00:00+00:00,233000.0,3.0,2.0,1710.0,4697,1.5,0
8562750320,2014-11-10 00:00:00+00:00,580500.0,3.0,2.5,2320.0,3980,2,0
7589200193,2014-11-10 00:00:00+00:00,535000.0,3.0,1.0,1090.0,3000,1.5,0
2078500320,2014-06-20 00:00:00+00:00,605000.0,4.0,2.5,2620.0,7553,2,0
7766200013,2014-08-11 00:00:00+00:00,775000.0,4.0,2.25,4220.0,24186,1,0
9478500640,2014-08-19 00:00:00+00:00,292500.0,4.0,2.5,2250.0,4495,2,0
9558200045,2014-08-28 00:00:00+00:00,289000.0,3.0,1.75,1260.0,8400,1,0
8820901275,2014-06-10 00:00:00+00:00,571000.0,4.0,2.0,2750.0,7807,1.5,0


In [39]:
test_data['11rss'].sum()

226568089092808.12

In [43]:
test_data['22rss'].sum()

224368799993634.7

In [44]:
test_data['33rss'].sum()

255723991553976.2